<a href="https://colab.research.google.com/github/soujanya-vattikolla/MongoDB-for-Python-Developers-/blob/main/Chapter4%20Resiliency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connection Pooling**

* Connection pools allow for reuse of connections.
* Subsequent requests appear faster to the client.
* Default size of 100.

**Problem:**<br>

Which of the following are benefits of connection pooling?<br>

* New operations can be serviced with pre-existing connections, so a new connection doesn't have to be created each time.

  * This is the main benefit from using a connection pool. The overhead of creating a TCP connection often results in waiting time, but this waiting time can be avoided by reusing a connection.

* A large influx of operations can be handled more quickly with a pool of existing connections.

    * A common cause of application crashes is the inability of that application to deal with an influx of operations to perform. By creating a lot of available connections before they are needed, the application has the bandwidth to service as many requests as connections, without having to create any new connections.

**Ticket: Connection Pooling**<br>
Problem:<br>

Task<br>

For this ticket, you'll be required to modify the configuration of MongoClient to set the maximum size of the connection pool to 50 connections.<br>

The MongoClient in db.py is initialized in the get_db method.

In [ ]:
def get_db():
    db = getattr(g, "_database", None)
    MFLIX_DB_URI = current_app.config["MFLIX_DB_URI"]
    if db is None:
        db = g._database = MongoClient(
            MFLIX_DB_URI,
            # here's where we set the maximum connection pool size to 50 connections
            maxPoolSize=50
        )["sample_mflix"]
    return db

**Robust Client Configuration**

* Always use connection pooling.
* Always specify a wtimeout with majority writes.
* Always handle serverSelectionTimeout errors.

**Problem:**<br>

When should you set a wtimeout?<br>

When our application is using a Write Concern more durable than w: 1.<br>

The primary reason to use a wtimeout is because by default, when using Write Concern more durable than w: 1, there is no wtimeout, so the server will wait indefinitely for operations to complete.<br>

Our application can use wtimeout to put a time limit on how long the server waits before a Write Concern is satisfied.

**Ticket: Timeouts**<br>
Problem:<br>

Task<br>

For this ticket, you'll be required to modify the connection information for MongoClient to set a write timeout of 2500 milliseconds.<br>

The MongoClient in db.py is initialized in the get_db method. 

In [ ]:
def get_db():
    """
    Configuration method to return db instance
    """
    db = getattr(g, "_database", None)
    MFLIX_DB_URI = current_app.config["MFLIX_DB_URI"]
    if db is None:
        db = g._database = MongoClient(
            MFLIX_DB_URI,
            maxPoolSize=50,

            # here's where we set the wtimeout value to 2500 milliseconds
            wtimeout=2500
            
        )["sample_mflix"]
    return db

**Writes with Error Handling**


* DuplicateKeyError can occur on _id as well as fields in unique indexes.

**Ticket: Handling Errors**<br>
**Problem:**<br>

Task<br>

For this ticket, you'll be required to make the API more robust by handling exceptions. Specifically, what would happen should an incorrectly formatted _id be passed to get_movie in db.py?

In [ ]:
def get_movie(id):
    try:
        pipeline = [
            {
                "$match": {
                    "_id": ObjectId(id)
                }
            },
            {
                "$lookup": {
                    "from": "comments",
                    "localField": "_id",
                    "foreignField": "movie_id",
                    "as": "comments"
                }
            }
        ]
        movie = db.movies.aggregate(pipeline).next()
        movie["comments"] = sorted(
            movie.get("comments", []),
            key=lambda c: c.get("date"),
            reverse=True
        )
        return movie
    # this will handle "InvalidId" the same way as "StopIteration"
    except (InvalidId, StopIteration) as _:
        return None

**Ticket: Principle of Least Privilege**<br>
Problem:<br>

Task<br>

For this ticket, you'll be required to add a new user on your Atlas cluster for the MFlix application to connect with.<br>

The user should follow credentials:<br>

username: mflixAppUser<br>
password: mflixAppPwd<br>

This user should have the readWrite role on the sample_mflix database. Use Add Default Privileges to assign the user this specific role.<br>

After you have created this user, modify the SRV connection string in your configuration file so the application connects with the new username and password.

In [ ]:
To complete this ticket, you had to create a user that only has readWrite access to the mflix database only.
Then replace the authentication credentials, with this new user ones, in the MongoDB URI SRV string in your configuration file.

**Change Streams**

* Change streams can be opened against a collection.
* Tracks data changes in real time.
* Aggregation pipelines can be used to transform change event documents.

**Problem:**<br>

What of the following is true about Change Streams in Pymongo?<br>

* They can be used to log changes to a MongoDB collection.

    * As of MongoDB 4.0, Change Streams can also be used to log changes at a database, and even a cluster level.

*  They output cursors, which contain change event documents.

    * We can iterate through these cursors like any other Python iterable.

* They accept pipelines, which can be used to filter output from the change stream.

    * These pipelines accept a subset of the stages in an Aggregation query, such as $match, $project, and $addFields.